In [1]:
%matplotlib qt 

#import bezier 
from scipy import interpolate
import pygame
import matplotlib.pyplot as plt
import numpy as np
from bicyclemodel import draw_car_indep  #x,y,yaw,v
import matplotlib.cm as cm

def bezier_func(nodes,s_vals):
    eval = []
    for t in s_vals:
        evaluation = (1-t)**3*nodes[0] + 3*(1-t)**2*t*nodes[1]+3*(1-t)*t**2*nodes[2]+ t**3*nodes[3]

        eval.append(evaluation)
    eval = np.array(eval)
    #print(eval)
    return eval


def bezier_K(P,s_vals):
    eval = []
    for t in s_vals:
        B_d  = 3*(1-t)**2*(P[1]-P[0]) + 6*(1-t)*t*(P[2]-P[1]) + 3*t**2*(P[3]-P[2])
        B_dd = 6*(1-t)*(P[2]-2*P[1]-P[0]) + 6*t*(P[3]-2*P[2]-P[1])
        N = np.linalg.det(np.vstack((B_d,B_dd)).T)
        
        #N = np.absolute(np.cross(B_d,B_dd))
        absolute_B_d = np.absolute(B_d)
        D = ((np.hypot(absolute_B_d[0],absolute_B_d[1]))**3)
        evaluation = N/D


        #print(f"{B_d},{B_dd},{cross},{absol} ")
        eval.append(evaluation)
    return np.array(eval)

def calc_car_bezier(state1,state2,w1,w2):

    Point_1 = np.array([state1[0],state1[1]])
    Point_4 = np.array([state2[0],state2[1]])
    yaw1 = state1[2]
    yaw2 = state2[2]
    Point_2 = Point_1 + np.array([w1*np.cos(yaw1),w1*np.sin(yaw1)])
    Point_3 = Point_4 + np.array([-w2*np.cos(yaw2),-w2*np.sin(yaw2)])

    points = np.vstack((Point_1,Point_2,Point_3,Point_4))
    
    nodes = np.asfortranarray(points.T)
    #curve = bezier.Curve(nodes,degree=3)
    #curvepoints = curve.eval
    s_vals = np.linspace(0,1,300)
    #curve.plot(50,ax=plt)
    curve_points = bezier_func(points,s_vals)
    curvature = bezier_K(points,s_vals)
    
    #print(curvature)

    #plt.plot(curve_points.T[0],curve_points.T[1])

    return curve_points,curvature,points

state1 = np.array([0,0,0,80])
state2 = np.array([80,60,1.9,80])

fig,axs = plt.subplots()

axs.axis("equal")
draw_car_indep(state1,plot=axs,delta=0.0,drawvel=True)
draw_car_indep(state2,plot=axs,delta=0.0,drawvel=True)


#plot_spline(state1,state2,10,10,plot=plt)
weights = np.array([20,100])
plot_c_points = False

globmax_curv = 0
globmin_curv = 0 # technically negative curvatures exist but for this applications/visualisation only the abolute curvature is neccesary
for w1 in weights:
    for w2 in weights:

        curve_points,curvature,C_points = calc_car_bezier(state1,state2,w1,w2)
        curvature = np.absolute(curvature)
        max_curvature = np.amax(curvature)
        
        if max_curvature > globmax_curv:
            globmax_curv = max_curvature
            

norm = plt.Normalize(globmin_curv, globmax_curv)
for w1 in weights:
    for w2 in weights:
        
        curve_points,curvature,C_points = calc_car_bezier(state1,state2,w1,w2)
        curvature = np.absolute(curvature)
        max_curvature = np.amax(curvature)
        
        #plt.plot(curve_points.T[0],curve_points.T[1])
        labelinfo = f"max K:{np.round(max_curvature,decimals=2)}, w1:{w1}, w2:{w2}" 
        axs.scatter(curve_points.T[0],curve_points.T[1],s=7,c=curvature,cmap='jet',norm=norm,label=labelinfo)
        
        if plot_c_points:
            axs.plot(C_points.T[0],C_points.T[1],linestyle='dashed',color='black',)
            axs.scatter(C_points.T[0],C_points.T[1],color='black')

        tck,u = interpolate.splprep([curve_points.T[0],curve_points.T[1]],s=0)
        unew = np.arange(0,1,0.01)
        out = interpolate.splev(unew,tck) 
        ind = int(np.array(out).shape[1]/2)
        x = out[0][ind]
        y = out[1][ind]
        axs.text(x,y,f"{round(max_curvature,3)}",fontsize=7,bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   ))

cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap='jet'), ax=axs)
axs.axis('equal')
cbar.set_label("|Curvature|")
axs.set_title("Bezier Curve Testing")
axs.legend(scatterpoints=0,framealpha=0)

plt.show()


pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
fig,axs = plt.subplots()

axs.set_title("Bézier curve with visble control points")
w1 = 60
w2 = 50

curve_points,curvature,C_points = calc_car_bezier(state1,state2,w1,w2)
curvature = np.absolute(curvature)
#globmin_curv = 0
#globmax_curv = np.amax(curvature)
#norm = plt.Normalize(globmin_curv, globmax_curv)
#plt.plot(curve_points.T[0],curve_points.T[1])
axs.scatter(curve_points.T[0],curve_points.T[1],s=7,c=curvature,cmap='jet',norm=norm)
if True:
    axs.plot(C_points.T[0],C_points.T[1],linestyle='dashed',color='black',)
    axs.scatter(C_points.T[0],C_points.T[1],color='black')

axs.text(C_points[0][0],C_points[0][1]+3,"P1",fontsize=10)
axs.text(C_points[1][0],C_points[1][1]+3,"P2",fontsize=10)
axs.text(C_points[2][0],C_points[2][1]+3,"P3",fontsize=10)
axs.text(C_points[3][0]+3,C_points[3][1]-1,"P4",fontsize=10)

cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap='jet'), ax=axs)
axs.axis('equal')
cbar.set_label("|Curvature|")

plt.show()

In [33]:

state3 = np.array([0,0,0,0])


fig,axs = plt.subplots()

axs.axis("equal")
draw_car_indep(state3,plot=axs,delta=0.0,)
axs.annotate(s='',xy=(0.5+14,6.5),xytext=(0.5+14,6.5-3),arrowprops=dict(arrowstyle='<->'))
axs.text(14+1,6.5-1.5,"3")
axs.annotate(s='',xy=(14,6.5+0.5),xytext=(14-8,6.5+0.5),arrowprops=dict(arrowstyle='<->'))
axs.text(14-4,6.5+1,"8")
axs.annotate(s='',xy=(-10,5+0.5),xytext=(10,5+0.5),arrowprops=dict(arrowstyle='<->'))
axs.text(0,5+1,"20")
axs.annotate(s='',xy=(10+0.5,5),xytext=(10+0.5,-5),arrowprops=dict(arrowstyle='<->'))
axs.text(10+1,0,"10")

axs.annotate(s='',xy=(0,0),xytext=(14,-6.5),arrowprops=dict(arrowstyle='<->'))
axs.text(3,-1,"r = 15.9")

r = np.sqrt(14**2+6.5**2)
t = np.linspace(0,2*np.pi,100)
x=np.cos(t)*r
y=np.sin(t)*r
plt.plot(x,y,linestyle='dashed',color='black',linewidth=1.0)

plt.show()


C:\Users\sasha\AppData\Local\Temp/ipykernel_15044/612162763.py:8: MatplotlibDeprecationWarning: The 's' parameter of annotate() has been renamed 'text' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  axs.annotate(s='',xy=(0.5+14,6.5),xytext=(0.5+14,6.5-3),arrowprops=dict(arrowstyle='<->'))
C:\Users\sasha\AppData\Local\Temp/ipykernel_15044/612162763.py:10: MatplotlibDeprecationWarning: The 's' parameter of annotate() has been renamed 'text' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  axs.annotate(s='',xy=(14,6.5+0.5),xytext=(14-8,6.5+0.5),arrowprops=dict(arrowstyle='<->'))
C:\Users\sasha\AppData\Local\Temp/ipykernel_15044/612162763.py:12: MatplotlibDeprecationWarning: The 's' parameter of annotate() has been renamed 'text' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  axs.annotate(s='',xy=(-10,5+0.5),xytext=(10,5+0.5),arrowprops=dict(arrowstyle='<->'))
C